In [1]:
import csv
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import csv

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras, convert_refs_to_rgbs
from chromalab.maxbasis import MaxBasis
from chromalab.inks import InkGamut
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction
from chromalab.animation import Animation

objc[99009]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ac7c2e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2d06b26f0). One of the two will be used. Which one is undefined.
objc[99009]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ac7c2db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2d06b2718). One of the two will be used. Which one is undefined.
objc[99009]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ac7c2e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [5]:
tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis = MaxBasis(tetrachromat, verbose=True)
viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True) 

100%|██████████| 28/28 [00:00<00:00, 65.85it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 8990/8990 [00:19<00:00, 468.42it/s]


In [6]:
cmy_primaries_dict = {}
with open(f'../data/nix/PrintColors/000.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader): 
        if i == 4:
            color_data = np.array(row[33:],dtype=float)
            spectra = Spectra(data=color_data, wavelengths=wavelengths)
            cmy_primaries_dict["000"] = spectra

In [7]:
# Polyscope settings
viz.ps.set_ground_plane_mode("none")

# render all objects
viz.renderObjectsPS(mesh_alpha=0.8, lattice_alpha=1)
viz.renderQArrow()

# transform to correct dir
mat = viz._getTransformQUpDir()
viz.ps.get_surface_mesh("mesh").set_transform(mat)
viz.ps.get_surface_mesh("lattice").set_transform(mat)
viz.ps.get_surface_mesh("qarrow").set_transform(mat)

In [8]:
names = ["mesh", "lattice", "qarrow"]

In [9]:
# Ideal Printer Gamut
refs, discrete_points, rgbs, lines = maxbasis.getDiscreteRepresentation()
cvpy_idxs = [-2, -3, -4, -5]
collection = list(np.array(refs)[cvpy_idxs])


paper = cmy_primaries_dict["000"]  
gamut = InkGamut(collection, paper, illuminant=d65)
point_cloud, perc = gamut.get_point_cloud(tetrachromat, stepsize=0.5)
names += viz.renderPrinterGamutConvexHull("cvpy", point_cloud, mat=mat)

Generating point cloud: 1it [00:00, 2308.37it/s]


In [27]:
real_ink_locs = np.array([ (viz.HMatrix@maxbasis.cone_to_maxbasis@tetrachromat.observe(ref))[1:] for ref in collection])
real_ink_rgbs = np.array([ ref.to_rgb() for ref in collection])
names += viz.renderPointCloud("CVPY_inks", real_ink_locs, real_ink_rgbs, radius=0.018 * 2, mat=mat)


In [11]:
names += viz._getCoordBasis("cvpy_coord", (mat[:3, :3]@real_ink_locs.T).T)

In [12]:
all_inks = {}
with open(f'../data/nix/submittable-printing-library.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=',')
    for i, row in enumerate(spamreader):
        if i < 1 or (i > 44 and i < 52): continue            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        spectra = Spectra(data=color_data, wavelengths=wavelengths)
        all_inks[name] = spectra

In [ ]:
real_ink_locs = np.array([ (viz.HMatrix@maxbasis.cone_to_maxbasis@tetrachromat.observe(ref))[1:] for ref in submitted_gamut])
real_ink_rgbs = np.array([ ref.to_rgb() for ref in submitted_gamut])

names += viz.renderPointCloud("chosen_inks", real_ink_locs, real_ink_rgbs, radius=0.018, mat=mat)


In [21]:
all_inks = {}
with open(f'../data/nix/submittable-printing-library.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=',')
    for i, row in enumerate(spamreader):
        if i < 1 or (i > 44 and i < 52): continue            
        name = row[4]
        color_data = np.array(row[33:],dtype=float)
        spectra = Spectra(data=color_data, wavelengths=wavelengths)
        all_inks[name] = spectra

ink_spectras = list(all_inks.values())
ink_locs = np.array([ (viz.HMatrix@maxbasis.cone_to_maxbasis@tetrachromat.observe(ref))[1:] for ref in ink_spectras])
ink_rgbs = np.array([ ref.to_rgb() for ref in ink_spectras])

names += viz.renderPointCloud("inklib", ink_locs, ink_rgbs, radius=0.018, mat=mat)

In [13]:
submitted_gamut = [all_inks["Platinum violet"],  
              all_inks["Peach haze D"], 
              all_inks["Operation overlord orange N"],
              all_inks['Epson CMY 100']]

real_ink_locs = np.array([ (viz.HMatrix@maxbasis.cone_to_maxbasis@tetrachromat.observe(ref))[1:] for ref in submitted_gamut])
real_ink_rgbs = np.array([ ref.to_rgb() for ref in submitted_gamut])

names += viz.renderPointCloud("chosen_inks", real_ink_locs, real_ink_rgbs, radius=0.018, mat=mat)

paper = cmy_primaries_dict["000"]  
gamut = InkGamut(submitted_gamut, paper, illuminant=d65)
point_cloud, perc = gamut.get_point_cloud(tetrachromat, stepsize=0.05)
names += viz.renderPrinterGamut("real_ink", point_cloud, mat=mat)[0]

Generating point cloud: 2it [00:00,  7.80it/s]                       


In [14]:
CMY_gamut = [all_inks["Epson CMY 100"], all_inks["Epson CMY 010"], all_inks["Epson CMY 001"]]

real_ink_locs = np.array([ (viz.HMatrix@maxbasis.cone_to_maxbasis@tetrachromat.observe(ref))[1:] for ref in CMY_gamut])
real_ink_rgbs = np.array([ ref.to_rgb() for ref in CMY_gamut])
names += viz.renderPointCloud("CMY_points", real_ink_locs, real_ink_rgbs, radius=0.018, mat=mat)

gamut = InkGamut(CMY_gamut, paper, illuminant=d65)
point_cloud, perc = gamut.get_point_cloud(tetrachromat, stepsize=0.05)
names += viz.renderPrinterGamut("CMY_Gamut", point_cloud, mat=mat)[0]

Generating point cloud: 1it [00:00, 57.98it/s]


In [22]:
types =  ['surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh', 'point_cloud', 'surface_mesh', 'point_cloud', 'surface_mesh', 'point_cloud', 'surface_mesh', 'point_cloud']
map_name_type = {name:typ for name, typ in zip(names, types)}

anim = Animation(viz, map_name_type)

In [32]:
short_frames = 180//3
medium_frames = 180 * 2//3
long_frames = 180
r = 4
theta = 65
phi = 360
offset = 0


In [17]:
filename = "./outputs/CMY_Gamut.mp4"
anim.set_enabled('lattice', False)
anim.set_enabled('cvpy_printer_mesh', False)
fd = viz.openVideo(filename)
rotAroundZ, phi = anim.RotateAroundZ(long_frames, r, theta, [0, phi])
fadeOutMesh = anim.FadeOut("mesh", long_frames, [0.4, 0.8])
concatAll = anim.concatFns([rotAroundZ, fadeOutMesh])
offset = viz.renderVideo(concatAll, fd, long_frames, offset)
viz.closeVideo(fd)

RAYCAST_SPHERE
SPHERE_PROPAGATE_COLOR
SHADE_COLOR
LIGHT_MATCAP
RAYCAST_SPHERE
SPHERE_PROPAGATE_COLOR
SHADE_COLOR
LIGHT_MATCAP
RAYCAST_SPHERE
SPHERE_PROPAGATE_COLOR
SHADE_COLOR
LIGHT_MATCAP


ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 

In [19]:
filename = "./outputs/CVPY_Gamut.mp4"
anim.set_enabled('lattice', False)
anim.set_enabled('cvpy_printer_mesh', True)
anim.set_enabled('CMY_Gamut_printer_mesh', False)
anim.set_enabled("CMY_points", False)
anim.set_enabled("qarrow", False)
anim.set_enabled('CVPY_inks', True)
anim.set_enabled('cvpy_coord', True)
anim.set_transparency("cvpy_printer_mesh", 0.5)
anim.set_enabled('ink_lib', False)

anim.set_enabled('chosen_inks', False)
anim.set_enabled('real_ink_printer_mesh', False)

fd = viz.openVideo(filename)
rotAroundZ, phi = anim.RotateAroundZ(long_frames, r, theta, [0, phi])
fadeOutMesh = anim.FadeOut("mesh", long_frames, [0.4, 0.6])
concatAll = anim.concatFns([rotAroundZ, fadeOutMesh])
offset = viz.renderVideo(concatAll, fd, long_frames, offset)
viz.closeVideo(fd)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 

In [30]:
names

['mesh',
 'lattice',
 'qarrow',
 'cvpy_printer_mesh',
 'CVPY_inks',
 'cvpy_coord',
 'chosen_inks',
 'real_ink_printer_mesh',
 'CMY_points',
 'CMY_Gamut_printer_mesh',
 'inklib',
 'CVPY_inks']

In [41]:
offset = 0

In [42]:
filename = "./outputs/library_of_inks.mp4"
anim.set_enabled('lattice', False)
anim.set_enabled('cvpy_printer_mesh', False)
anim.set_enabled('CMY_Gamut_printer_mesh', False)
anim.set_enabled("CMY_points", False)
anim.set_enabled("qarrow", False)
anim.set_enabled('CVPY_inks', True)
anim.set_transparency("cvpy_printer_mesh", 0.5)

anim.set_enabled('inklib', True)
anim.set_enabled('chosen_inks', False)
anim.set_enabled('real_ink_printer_mesh', False)
last_phi = 270
fd = viz.openVideo(filename)
rotAroundZ, phi = anim.RotateAroundZ(short_frames, r, theta, [0, last_phi/3])
fadeOutInks = anim.FadeOut("inklib", short_frames, [0.4, 0.6], removeBefore=5)
offset = viz.renderVideo(anim.concatFns([rotAroundZ, fadeOutInks]), fd, short_frames, offset)

rotAroundZ, phi = anim.RotateAroundZ(short_frames, r, theta, [phi, last_phi/3 * 2])
anim.set_enabled('real_ink_printer_mesh', True)
anim.set_enabled('chosen_inks', True)
fadeInGamut = anim.FadeIn("real_ink_printer_mesh", short_frames, [0.4, 0.8])
fadeInInks = anim.FadeIn("chosen_inks", short_frames, [0.4, 0.8])
offset = viz.renderVideo(anim.concatFns([rotAroundZ, fadeInGamut, fadeInInks]), fd, short_frames, offset)

rotAroundZ, phi = anim.RotateAroundZ(medium_frames, r, theta, [phi, 360])
offset = viz.renderVideo(anim.concatFns([rotAroundZ, fadeInGamut, fadeInInks]), fd, medium_frames, offset)

viz.closeVideo(fd)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 

In [ ]:
filename = "./outputs/chosen_ink_gamut.mp4"
anim.set_transparency('mesh', 0.5)
anim.set_enabled('qarrow', False)
anim.set_enabled('chosen_inks', True)
anim.set_enabled('real_ink_printer_mesh', True)
anim.set_enabled('lattice', False)
anim.set_enabled('cvpy_printer_mesh', False)
anim.set_enabled('CMY_Gamut_printer_mesh', False)
anim.set_enabled("CMY_points", False)

fd = viz.openVideo(filename)
rotAroundZ, phi = anim.RotateAroundZ(long_frames, r, theta, [0, phi])
offset = viz.renderVideo(rotAroundZ, fd, long_frames, offset)
viz.closeVideo(fd)